In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

# Load the cleaned dataset
df = pd.read_csv('/Users/jadavravi/Desktop/DA Projects/Myntra /Myntra_DataCleaning.csv')

print("Cleaned dataset shape:", df.shape)
print("\nData types:")
print(df.dtypes)

# Convert numeric columns to proper data types
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['MRP'] = pd.to_numeric(df['MRP'], errors='coerce')
df['discount_percent'] = pd.to_numeric(df['discount_percent'], errors='coerce')
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
df['number_of_ratings'] = pd.to_numeric(df['number_of_ratings'], errors='coerce')

print("\nData types after conversion:")
print(df.dtypes)

# 1. Summary statistics
print("\n=== SUMMARY STATISTICS ===")
summary_stats = df.describe().round(2)
print(summary_stats)

# 2. Frequency counts
print("\n=== FREQUENCY COUNTS ===")

# Products per brand
brand_counts = df['brand_name'].value_counts()
print("\nTop 20 brands by product count:")
print(brand_counts.head(20))

# Most common fits/descriptions from pants_description
# Extract common fit types
df['fit_type'] = df['pants_description'].str.extract(r'(Slim|Regular|Relaxed|Loose|Skinny|Straight|Anti|Tapered)', flags=re.IGNORECASE)[0]
df['fit_type'] = df['fit_type'].str.title().fillna('Other')

fit_counts = df['fit_type'].value_counts()
print("\nFit type distribution:")
print(fit_counts)

# 3. Distribution analysis
print("\n=== DISTRIBUTION ANALYSIS ===")

# Price distribution
price_stats = df['price'].describe()
print("\nPrice distribution:")
print(f"Mean: ₹{price_stats['mean']:.2f}")
print(f"Median: ₹{price_stats['50%']:.2f}")
print(f"Std: ₹{price_stats['std']:.2f}")
print(f"Min: ₹{price_stats['min']:.2f}")
print(f"Max: ₹{price_stats['max']:.2f}")

# Ratings distribution
rating_stats = df['ratings'].describe()
print("\nRatings distribution:")
print(f"Mean: {rating_stats['mean']:.2f}")
print(f"Median: {rating_stats['50%']:.2f}")
print(f"Std: {rating_stats['std']:.2f}")

# Discount distribution
discount_stats = df['discount_percent'].describe()
print("\nDiscount distribution:")
print(f"Mean: {discount_stats['mean']:.2%}")
print(f"Median: {discount_stats['50%']:.2%}")
print(f"Max: {discount_stats['max']:.2%}")

# 4. Relationship analysis
print("\n=== RELATIONSHIP ANALYSIS ===")

# Correlation matrix
correlation_matrix = df[['price', 'MRP', 'discount_percent', 'ratings', 'number_of_ratings']].corr()
print("\nCorrelation matrix:")
print(correlation_matrix.round(3))

# Average price by brand
average_price_by_brand = df.groupby('brand_name')['price'].agg(['mean', 'count']).round(2)
average_price_by_brand = average_price_by_brand.sort_values('mean', ascending=False)
print("\nTop 10 brands by average price:")
print(average_price_by_brand.head(10))

# Average rating by brand
average_rating_by_brand = df.groupby('brand_name')['ratings'].agg(['mean', 'count']).round(2)
average_rating_by_brand = average_rating_by_brand.sort_values('mean', ascending=False)
print("\nTop 10 brands by average rating:")
print(average_rating_by_brand.head(10))

# Return summary statistics for further analysis
summary_stats

Cleaned dataset shape: (35073, 7)

Data types:
brand_name            object
pants_description     object
price                float64
MRP                  float64
discount_percent     float64
ratings              float64
number_of_ratings    float64
dtype: object

Data types after conversion:
brand_name            object
pants_description     object
price                float64
MRP                  float64
discount_percent     float64
ratings              float64
number_of_ratings    float64
dtype: object

=== SUMMARY STATISTICS ===
          price       MRP  discount_percent   ratings  number_of_ratings
count  35073.00  35073.00          35073.00  35073.00           35073.00
mean    1618.15   3277.91              0.47      3.98             103.71
std     1758.24   2572.05              0.18      0.45             520.46
min      337.00    499.00              0.01      1.00               5.00
25%      899.00   2399.00              0.38      3.80              14.00
50%     1418.00   2999.

,price,MRP,discount_percent,ratings,number_of_ratings
count,35073.00,35073.00,35073.00,35073.00,35073.00
mean,1618.15,3277.91,0.47,3.98,103.71
std,1758.24,2572.05,0.18,0.45,520.46
min,337.00,499.00,0.01,1.00,5.00
25%,899.00,2399.00,0.38,3.80,14.00
50%,1418.00,2999.00,0.50,4.00,31.00
75%,1829.00,3699.00,0.60,4.20,75.00
max,54000.00,72000.00,1.00,5.00,30700.00


In [6]:
import pandas as pd
import numpy as np
import re

# Load the cleaned dataset
df = pd.read_csv('/Users/jadavravi/Desktop/DA Projects/Myntra /Myntra_DataCleaning.csv')


# 1. Calculate absolute discount: MRP - price
df['absolute_discount'] = df['MRP'] - df['price']
print(f"\nAdded absolute discount column")

# 2. Create rating buckets
def get_rating_bucket(rating):
    if pd.isna(rating):
        return 'No Rating'
    elif rating >= 4.0:
        return 'High (4+)'
    elif rating >= 3.0:
        return 'Medium (3-4)'
    else:
        return 'Low (<3)'

df['rating_bucket'] = df['ratings'].apply(get_rating_bucket)
print(f"Added rating bucket column")

# 3. Extract fit type from pants_description
def extract_fit_type(description):
    if pd.isna(description):
        return 'Unknown'
    
    description = str(description).lower()
    
    if 'slim' in description:
        return 'Slim Fit'
    elif 'regular' in description:
        return 'Regular Fit'
    elif 'relaxed' in description:
        return 'Relaxed Fit'
    elif 'loose' in description:
        return 'Loose Fit'
    elif 'skinny' in description:
        return 'Skinny Fit'
    elif 'straight' in description:
        return 'Straight Fit'
    elif 'tapered' in description:
        return 'Tapered Fit'
    elif 'anti' in description:
        return 'Anti Fit'
    else:
        return 'Other Fit'

df['fit_type'] = df['pants_description'].apply(extract_fit_type)
print(f"Added fit type column")

# 4. Create value_for_money score
df['value_for_money'] = df['ratings'] * df['discount_percent']
print(f"Added value for money score column")

# 5. Additional useful features
# Price to MRP ratio
df['price_mrp_ratio'] = df['price'] / df['MRP']

# Discount percentage category
def get_discount_category(discount):
    if pd.isna(discount):
        return 'No Discount'
    elif discount >= 0.5:
        return 'High Discount (50%+)'
    elif discount >= 0.3:
        return 'Medium Discount (30-50%)'
    elif discount >= 0.1:
        return 'Low Discount (10-30%)'
    else:
        return 'Minimal Discount (<10%)'

df['discount_category'] = df['discount_percent'].apply(get_discount_category)

print(f"\nNew columns added:")
print("- absolute_discount: MRP - price")
print("- rating_bucket: Categorical rating groups")
print("- fit_type: Extracted from pants description")
print("- value_for_money: ratings * discount_percent")
print("- price_mrp_ratio: price / MRP")
print("- discount_category: Categorical discount groups")

print(f"\nDataset shape after feature engineering: {df.shape}")
print(f"\nNew column names: {list(df.columns)}")

# Show sample of new features
print(f"\nSample of new features:")
print(df[['brand_name', 'price', 'MRP', 'absolute_discount', 'ratings', 'rating_bucket', 
          'discount_percent', 'discount_category', 'fit_type', 'value_for_money']].head(10))

# Return the enhanced dataset
df


Added absolute discount column
Added rating bucket column
Added fit type column
Added value for money score column

New columns added:
- absolute_discount: MRP - price
- rating_bucket: Categorical rating groups
- fit_type: Extracted from pants description
- value_for_money: ratings * discount_percent
- price_mrp_ratio: price / MRP
- discount_category: Categorical discount groups

Dataset shape after feature engineering: (35073, 13)

New column names: ['brand_name', 'pants_description', 'price', 'MRP', 'discount_percent', 'ratings', 'number_of_ratings', 'absolute_discount', 'rating_bucket', 'fit_type', 'value_for_money', 'price_mrp_ratio', 'discount_category']

Sample of new features:
       brand_name   price     MRP  absolute_discount  ratings rating_bucket  \
0           Wrogn  1374.0  2499.0             1125.0      4.2     High (4+)   
1  Flying Machine  1829.0  2999.0             1170.0      4.6     High (4+)   
2        Roadster   974.0  2499.0             1525.0      3.6  Medium

,brand_name,pants_description,price,MRP,discount_percent,ratings,number_of_ratings,absolute_discount,rating_bucket,fit_type,value_for_money,price_mrp_ratio,discount_category
0,Wrogn,Men Loose Fit Cotton Jeans,1374.0,2499.0,0.45,4.2,57.0,1125.0,High (4+),Loose Fit,1.890,0.549820,Medium Discount (30-50%)
1,Flying Machine,Men Slim Fit Jeans,1829.0,2999.0,0.39,4.6,5.0,1170.0,High (4+),Slim Fit,1.794,0.609870,Medium Discount (30-50%)
2,Roadster,Men Pure Cotton Jeans,974.0,2499.0,0.61,3.6,1100.0,1525.0,Medium (3-4),Other Fit,2.196,0.389756,High Discount (50%+)
3,Bene Kleed,Relaxed Fit Denim Jeans,873.0,2299.0,0.62,4.0,4800.0,1426.0,High (4+),Relaxed Fit,2.480,0.379730,High Discount (50%+)
4,Levis,Men 511 Slim Fit Jeans,1478.0,2899.0,0.49,4.3,264.0,1421.0,High (4+),Slim Fit,2.107,0.509831,Medium Discount (30-50%)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35068,United Colors Of Benetton,Men Slim Fit Mid-Rise Jeans,949.0,3049.0,0.21,3.8,6.0,2100.0,Medium (3-4),Slim Fit,0.798,0.311250,Low Discount (10-30%)
35069,Here&Now,Men Slim Fit Jeans,759.0,1899.0,0.60,4.0,63.0,1140.0,High (4+),Slim Fit,2.400,0.399684,High Discount (50%+)
35070,Pepe Jeans,Men Vapour Slim Fit Jeans,759.0,1899.0,0.60,4.0,63.0,1140.0,High (4+),Slim Fit,2.400,0.399684,High Discount (50%+)
35071,Pepe Jeans,Men Slim Fit Jeans,759.0,1899.0,0.60,4.0,63.0,1140.0,High (4+),Slim Fit,2.400,0.399684,High Discount (50%+)


In [7]:
import pandas as pd

# Load the feature engineered dataset
df
print("Analyzing discount categories relationship with ratings and sales volume...")

# 1. Group by discount category and calculate metrics
discount_analysis = df.groupby('discount_category').agg({
    'ratings': ['mean', 'count', 'std'],
    'number_of_ratings': ['mean', 'sum', 'std'],
    'price': 'mean',
    'absolute_discount': 'mean',
    'value_for_money': 'mean'
}).round(2)

discount_analysis.columns = ['_'.join(col).strip() for col in discount_analysis.columns.values]
discount_analysis = discount_analysis.rename(columns={
    'ratings_mean': 'avg_rating',
    'ratings_count': 'product_count',
    'ratings_std': 'rating_std',
    'number_of_ratings_mean': 'avg_ratings_count',
    'number_of_ratings_sum': 'total_ratings',
    'number_of_ratings_std': 'ratings_count_std',
    'price_mean': 'avg_price',
    'absolute_discount_mean': 'avg_absolute_discount',
    'value_for_money_mean': 'avg_value_for_money'
})

print("\n=== DISCOUNT CATEGORY ANALYSIS ===")
print(discount_analysis)

# 2. Statistical insights
print("\n=== KEY INSIGHTS ===")

# Correlation between discount percentage and ratings
corr_discount_rating = df['discount_percent'].corr(df['ratings'])
print(f"Correlation between discount % and ratings: {corr_discount_rating:.3f}")

# Correlation between discount percentage and number of ratings
corr_discount_volume = df['discount_percent'].corr(df['number_of_ratings'])
print(f"Correlation between discount % and number of ratings: {corr_discount_volume:.3f}")

# Most popular discount category by product count
most_products_category = discount_analysis['product_count'].idxmax()
print(f"Most common discount category by product count: {most_products_category}")

# Highest rated discount category
highest_rated_category = discount_analysis['avg_rating'].idxmax()
print(f"Highest rated discount category: {highest_rated_category}")

# Highest sales volume category
highest_volume_category = discount_analysis['total_ratings'].idxmax()
print(f"Highest sales volume category: {highest_volume_category}")

# Best value for money category
best_value_category = discount_analysis['avg_value_for_money'].idxmax()
print(f"Best value for money category: {best_value_category}")

# Return the analysis results
discount_analysis

Analyzing discount categories relationship with ratings and sales volume...

=== DISCOUNT CATEGORY ANALYSIS ===
                          avg_rating  product_count  rating_std  \
discount_category                                                 
High Discount (50%+)            3.95          18849        0.45   
Low Discount (10-30%)           3.99           3786        0.43   
Medium Discount (30-50%)        4.01          10257        0.45   
Minimal Discount (<10%)         3.99           2181        0.49   

                          avg_ratings_count  total_ratings  ratings_count_std  \
discount_category                                                               
High Discount (50%+)                 115.21      2171637.0             638.78   
Low Discount (10-30%)                 90.42       342322.0             409.49   
Medium Discount (30-50%)              92.82       952007.0             328.96   
Minimal Discount (<10%)               78.57       171360.0             164.75   

,avg_rating,product_count,rating_std,avg_ratings_count,total_ratings,ratings_count_std,avg_price,avg_absolute_discount,avg_value_for_money
discount_category,,,,,,,,,
High Discount (50%+),3.95,18849,0.45,115.21,2171637.0,638.78,1291.73,1942.03,2.39
Low Discount (10-30%),3.99,3786,0.43,90.42,342322.0,409.49,2246.35,1334.36,0.81
Medium Discount (30-50%),4.01,10257,0.45,92.82,952007.0,328.96,2145.10,1478.39,1.59
Minimal Discount (<10%),3.99,2181,0.49,78.57,171360.0,164.75,870.59,638.15,0.30


In [ ]:
import pandas as pd


df

print("Creating data visualizations...")

# 1. Bar chart data: Top 20 brands with most products
brand_counts = df['brand_name'].value_counts().head(20)
brand_chart_data = pd.DataFrame({
    'Brand': brand_counts.index,
    'Product_Count': brand_counts.values
})

# 2. Pie chart data: Distribution of fits
fit_counts = df['fit_type'].value_counts()
fit_chart_data = pd.DataFrame({
    'Fit_Type': fit_counts.index,
    'Count': fit_counts.values
})

# 3. Box plot data: Price distribution by top 10 brands
top_brands = df['brand_name'].value_counts().head(10).index
brand_price_data = df[df['brand_name'].isin(top_brands)][['brand_name', 'price']]

# 4. Scatter plot data: Rating vs Price
scatter_data = df[['price', 'ratings', 'brand_name', 'pants_description', 'discount_percent', 'number_of_ratings']]

# 5. Correlation matrix
correlation_matrix = df[['price', 'MRP', 'discount_percent', 'ratings', 'number_of_ratings', 'absolute_discount', 'value_for_money']].corr()

# 6. Discount % by fit type
discount_fit_data = df[['fit_type', 'discount_percent']]

# 7. Value for money by brand
value_by_brand = df.groupby('brand_name')['value_for_money'].mean().sort_values(ascending=False).head(15)
value_data = pd.DataFrame({
    'Brand': value_by_brand.index,
    'Value_Score': value_by_brand.values
})

print("Visualization data prepared successfully!")
print("\nAvailable datasets for visualization:")
print("1. Brand product counts - Top 20 brands")
print("2. Fit type distribution")
print("3. Price distribution by top 10 brands")
print("4. Rating vs Price scatter data")
print("5. Correlation matrix")
print("6. Discount % by fit type")
print("7. Value for money by top 15 brands")

print("\nSample of brand product counts:")
print(brand_chart_data.head())

print("\nSample of fit type distribution:")
print(fit_chart_data.head())

print("\nCorrelation matrix:")
print(correlation_matrix.round(3))

# Return the visualization datasets
visualization_data = {
    'brand_counts': brand_chart_data,
    'fit_counts': fit_chart_data,
    'correlation_matrix': correlation_matrix
}

visualization_data

Creating data visualizations...
Visualization data prepared successfully!

Available datasets for visualization:
1. Brand product counts - Top 20 brands
2. Fit type distribution
3. Price distribution by top 10 brands
4. Rating vs Price scatter data
5. Correlation matrix
6. Discount % by fit type
7. Value for money by top 15 brands

Sample of brand product counts:
                       Brand  Product_Count
0  United Colors Of Benetton           3300
1             Flying Machine           2576
2                   Roadster           1796
3                     Spykar           1149
4                      Wrogn           1101

Sample of fit type distribution:
      Fit_Type  Count
0     Slim Fit  12006
1    Other Fit   8809
2  Relaxed Fit   3596
3   Skinny Fit   3162
4  Regular Fit   2558

Correlation matrix:
                   price    MRP  discount_percent  ratings  number_of_ratings  \
price              1.000  0.944            -0.162    0.087             -0.035   
MRP                0.

{'brand_counts':                         Brand  Product_Count
 0   United Colors Of Benetton           3300
 1              Flying Machine           2576
 2                    Roadster           1796
 3                      Spykar           1149
 4                       Wrogn           1101
 5   U.S. Polo Assn. Denim Co.            850
 6                      V-Mart            800
 7                    Glitchez            784
 8                  Pepe Jeans            659
 9                     Linaria            654
 10                      Mufti            621
 11         Calvin Klein Jeans            590
 12             Mast & Harbour            584
 13                      Levis            566
 14               Jack & Jones            493
 15                   Here&Now            483
 16                Being Human            428
 17                 Highlander            417
 18       Louis Philippe Jeans            414
 19                    Comfits            401,
 'fit_counts':   

In [9]:
import pandas as pd
import numpy as np

# Load the feature engineered dataset
df 

print("Extracting actionable insights from the analysis...")

# 1. Best value-for-money brands (high rating + high discount)
value_analysis = df.groupby('brand_name').agg({
    'value_for_money': 'mean',
    'ratings': 'mean',
    'discount_percent': 'mean',
    'number_of_ratings': 'sum',
    'price': 'mean'
}).round(3)

value_analysis = value_analysis.sort_values('value_for_money', ascending=False)

print("\n=== TOP 10 BEST VALUE-FOR-MONEY BRANDS ===")
print("(High ratings combined with good discounts)")
print(value_analysis.head(10))

# 2. Brands with high discounts but poor ratings
high_discount_low_rating = df.groupby('brand_name').agg({
    'discount_percent': 'mean',
    'ratings': 'mean',
    'number_of_ratings': 'sum'
}).round(3)

# Filter for brands with high discount (>40%) but low rating (<3.5)
problem_brands = high_discount_low_rating[
    (high_discount_low_rating['discount_percent'] > 0.4) & 
    (high_discount_low_rating['ratings'] < 3.5)
].sort_values('discount_percent', ascending=False)

print("\n=== BRANDS WITH HIGH DISCOUNTS BUT POOR RATINGS ===")
print("(Potential quality issues despite deep discounts)")
print(problem_brands)

# 3. Fit types performance analysis
fit_performance = df.groupby('fit_type').agg({
    'ratings': 'mean',
    'number_of_ratings': 'sum',
    'price': 'mean',
    'discount_percent': 'mean',
    'value_for_money': 'mean'
}).round(3).sort_values('ratings', ascending=False)

print("\n=== FIT TYPE PERFORMANCE ANALYSIS ===")
print("(Ranked by average rating)")
print(fit_performance)

# 4. Relationship between number of ratings and rating score - Alternative approach
# Create rating volume categories manually
def get_rating_volume_category(num_ratings):
    if num_ratings <= 10:
        return '0-10'
    elif num_ratings <= 50:
        return '10-50'
    elif num_ratings <= 100:
        return '50-100'
    elif num_ratings <= 500:
        return '100-500'
    elif num_ratings <= 1000:
        return '500-1000'
    else:
        return '1000+'

df['rating_volume_category'] = df['number_of_ratings'].apply(get_rating_volume_category)

rating_volume_stats = df.groupby('rating_volume_category').agg({
    'ratings': 'mean',
    'number_of_ratings': 'count'
}).round(3)

print("\n=== RATING VOLUME vs RATING SCORE ===")
print("(Does popularity correlate with quality?)")
print(rating_volume_stats)

# 5. Premium vs Budget brand analysis
# Define premium brands as those with average price > ₹2000
brand_pricing = df.groupby('brand_name')['price'].mean().sort_values(ascending=False)
premium_brands = brand_pricing[brand_pricing > 2000].index
budget_brands = brand_pricing[brand_pricing < 1000].index

premium_performance = df[df['brand_name'].isin(premium_brands)].agg({
    'ratings': 'mean',
    'discount_percent': 'mean',
    'value_for_money': 'mean'
}).round(3)

budget_performance = df[df['brand_name'].isin(budget_brands)].agg({
    'ratings': 'mean',
    'discount_percent': 'mean',
    'value_for_money': 'mean'
}).round(3)

print("\n=== PREMIUM vs BUDGET BRAND COMPARISON ===")
print("Premium brands (avg price > ₹2000):")
print(premium_performance)
print("\nBudget brands (avg price < ₹1000):")
print(budget_performance)

# 6. Key actionable insights
print("\n=== ACTIONABLE INSIGHTS ===")

# Best value brands
top_value_brand = value_analysis.index[0]
top_value_score = value_analysis['value_for_money'].iloc[0]
print(f"1. Best value brand: {top_value_brand} (Value score: {top_value_score})")

# Highest rated fit type
top_fit = fit_performance.index[0]
top_fit_rating = fit_performance['ratings'].iloc[0]
print(f"2. Highest rated fit type: {top_fit} (Avg rating: {top_fit_rating})")

# Rating vs volume insight
high_volume_rating = rating_volume_stats.loc['1000+', 'ratings']
low_volume_rating = rating_volume_stats.loc['0-10', 'ratings']
print(f"3. High volume products (1000+ ratings) avg rating: {high_volume_rating}")
print(f"   Low volume products (0-10 ratings) avg rating: {low_volume_rating}")

# Premium vs budget insight
premium_rating = premium_performance['ratings']
budget_rating = budget_performance['ratings']
print(f"4. Premium brands avg rating: {premium_rating} vs Budget brands: {budget_rating}")

# Return the comprehensive insights analysis
insights_summary = {
    'value_analysis': value_analysis.head(10),
    'problem_brands': problem_brands,
    'fit_performance': fit_performance,
    'rating_volume_stats': rating_volume_stats,
    'premium_vs_budget': {
        'premium': premium_performance,
        'budget': budget_performance
    }
}

insights_summary

Extracting actionable insights from the analysis...

=== TOP 10 BEST VALUE-FOR-MONEY BRANDS ===
(High ratings combined with good discounts)
                 value_for_money  ratings  discount_percent  \
brand_name                                                    
Hauwer                     3.264    4.060             0.804   
Hardsoda                   3.151    4.328             0.728   
Nuevosdamas                3.150    4.200             0.750   
Angelfab                   3.010    3.810             0.790   
American Archer            2.998    4.283             0.700   
Maahit                     2.925    3.800             0.770   
Color Hunt                 2.910    4.250             0.685   
Tci                        2.904    4.002             0.726   
Altiva                     2.899    4.262             0.680   
Dillinger                  2.890    3.950             0.732   

                 number_of_ratings     price  
brand_name                                    
Hauwer   

{'value_analysis':                  value_for_money  ratings  discount_percent  \
 brand_name                                                    
 Hauwer                     3.264    4.060             0.804   
 Hardsoda                   3.151    4.328             0.728   
 Nuevosdamas                3.150    4.200             0.750   
 Angelfab                   3.010    3.810             0.790   
 American Archer            2.998    4.283             0.700   
 Maahit                     2.925    3.800             0.770   
 Color Hunt                 2.910    4.250             0.685   
 Tci                        2.904    4.002             0.726   
 Altiva                     2.899    4.262             0.680   
 Dillinger                  2.890    3.950             0.732   
 
                  number_of_ratings     price  
 brand_name                                    
 Hauwer                       282.0   931.200  
 Hardsoda                     733.0   734.920  
 Nuevosdamas        

In [10]:
import pandas as pd

# Load your full dataset
df 
# === CLEANING (Light) ===
# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Drop rows with missing crucial data
df.dropna(subset=['price', 'mrp', 'ratings', 'number_of_ratings'], inplace=True)

# Ensure correct data types
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['mrp'] = pd.to_numeric(df['mrp'], errors='coerce')
df['discount_percent'] = pd.to_numeric(df['discount_percent'], errors='coerce')
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
df['number_of_ratings'] = pd.to_numeric(df['number_of_ratings'], errors='coerce')

# === FEATURE ENGINEERING ===
df['absolute_discount'] = df['mrp'] - df['price']
df['price_range'] = pd.cut(df['price'], bins=[0, 1000, 1500, 2000, float('inf')],
                           labels=['Below ₹1000', '₹1000–₹1500', '₹1501–₹2000', '₹2001 and above'])

# === 1. Price Range vs Ratings ===
price_rating = df.groupby('price_range').agg(
    avg_rating=('ratings', 'mean'),
    avg_discount=('discount_percent', 'mean'),
    count=('brand_name', 'count')
).reset_index()

print("\n📊 Average Rating & Discount by Price Range:\n")
print(price_rating.sort_values('price_range'))

# === 2. Brands That Need More Promotions ===
brand_stats = df.groupby('brand_name').agg(
    avg_discount=('discount_percent', 'mean'),
    avg_rating=('ratings', 'mean'),
    num_ratings=('number_of_ratings', 'sum'),
    count=('brand_name', 'count')
).sort_values(by='num_ratings', ascending=False).reset_index()

promo_brands = brand_stats[
    (brand_stats['avg_discount'] < 0.45) &
    (brand_stats['avg_rating'] >= 4.0) &
    (brand_stats['num_ratings'] > 25000)
]

print("\n🚀 Brands That May Benefit From More Promotions:")
display(promo_brands.sort_values(by='num_ratings', ascending=False))
# Example logic to filter:
low_discount_high_rating = brand_stats[(brand_stats['avg_discount'] < 0.45) & (brand_stats['avg_rating'] >= 4.0)]

print("\n🚀 Brands That May Benefit From More Promotions:\n")
print(low_discount_high_rating[['brand_name', 'avg_discount', 'avg_rating', 'num_ratings']])

# === 3. Top-Rated Products to Feature ===
top_rated = df[(df['ratings'] >= 4.2) & (df['number_of_ratings'] >= 100)].sort_values(
    ['ratings', 'number_of_ratings'], ascending=[False, False])

print("\n🌟 Top-Rated Products to Feature on Homepage:\n")
print(top_rated[['brand_name', 'pants_description', 'ratings', 'number_of_ratings', 'price']].head(10))

# === 4. High Price & Low Rating Products to Review ===
low_rating_high_price = df[(df['price'] > 1500) & (df['ratings'] < 3.8)]

print("\n⚠️ High-Priced, Low-Rated Products (Possible Issues):\n")
print(low_rating_high_price[['brand_name', 'pants_description', 'price', 'ratings', 'number_of_ratings']].head(10))



📊 Average Rating & Discount by Price Range:

       price_range  avg_rating  avg_discount  count
0      Below ₹1000    3.877339      0.510693  10825
1      ₹1000–₹1500    3.993987      0.512432   8432
2      ₹1501–₹2000    4.024327      0.448060   9023
3  ₹2001 and above    4.044723      0.373465   6793

🚀 Brands That May Benefit From More Promotions:

                    brand_name  avg_discount  avg_rating  num_ratings
10          Calvin Klein Jeans      0.440231    4.019661      68316.0
11   U.S. Polo Assn. Denim Co.      0.413017    4.096588      52583.0
15                     Linaria      0.102602    4.002141      46447.0
20                     Comfits      0.091280    4.008479      33001.0
22        Louis Philippe Jeans      0.355663    4.046618      28821.0
..                         ...           ...         ...          ...
405                     Sparky      0.200000    4.100000          9.0
407                       Nike      0.100000    5.000000          6.0
409           

/var/folders/q3/4nvjm84d4gsdzws2s870x3h00000gn/T/ipykernel_2531/2775658682.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  price_rating = df.groupby('price_range').agg(


✅ Final Business Recommendations (Data-Driven)
🎯 1. Optimal Price Range Based on Ratings
    
| Price Range         | Avg Rating | Avg Discount | Products |
| ------------------- | ---------- | ------------ | -------- |
| **Below ₹1000**     | 3.88       | 51.1%        | 10,825   |
| **₹1000–₹1500**     | 3.99       | 51.2%        | 8,432    |
| **₹1501–₹2000**     | 4.02       | 44.8%        | 9,023    |
| **₹2001 and above** | **4.04**   | **37.3%**    | 6,793    |

✅ Recommendation:

Target the ₹2000+ segment for high-value customers — best ratings despite lowest discounts.

Products in ₹1500–₹2000 have strong balance of good ratings and healthy discounts — great for mass appeal.

Consider improving quality or experience of < ₹1000 segment — rating dip may indicate quality concerns.

🏷️ 2. Brands That Should Be Promoted More

Filtered brands:

Avg Rating ≥ 4.0

Discount ≤ 45%

High engagement (number_of_ratings > 25K)

| Brand                     | Avg Discount | Avg Rating | Total Ratings |
| ------------------------- | ------------ | ---------- | ------------- |
| **Calvin Klein Jeans**    | 44.0%        | 4.02       | 68,316        |
| **U.S. Polo Assn. Denim** | 41.3%        | 4.10       | 52,583        |
| **Louis Philippe Jeans**  | 35.6%        | 4.05       | 28,821        |
| **Comfits**               | **9.1%**     | 4.01       | 33,001        |
| **Linaria**               | **10.3%**    | 4.00       | 46,447        |


✅ Recommendation:

Increase discount campaigns for Linaria and Comfits — high ratings and high reach, but very low discounts, which might be affecting conversion.

Promote U.S. Polo and Calvin Klein Jeans on banners — already popular with good feedback, just need a nudge.

🌟 Top Products to Feature on Homepage

Top criteria:

Rating ≥ 4.5

Reviews ≥ 100
| Brand              | Description                   | Rating  | Ratings | Price       |
| ------------------ | ----------------------------- | ------- | ------- | ----------- |
| **Pepe Jeans**     | Stretchable Jeans             | 4.6     | 186     | ₹1699       |
| **Flying Machine** | Relaxed Fit Jeans             | 4.6     | 186     | ₹2309       |
| **John Pride**     | Plus Size Jogger Jeans        | 4.6     | 114     | ₹1949       |
| **Levis**          | 511 Slim Fit, 512 Tapered Fit | 4.5–4.6 | 100–586 | ₹1959–₹2250 |
| **Spykar**         | Slim Fit Mid-Rise             | 4.6     | 114     | ₹2349       |
                                                                                 
✅ Recommendation:

Homepage carousel or “Best Rated” section featuring Levi’s, Pepe, John Pride, and Flying Machine Relaxed Fit will likely boost CTR and conversions.

⚠️ High-Priced, Low-Rated Products to Investigate

Flagged criteria:

Price > ₹1500

Rating < 3.8

Ratings count > 50 (i.e., not anomalies)

| Brand          | Product                    | Price | Rating | Reviews |
| -------------- | -------------------------- | ----- | ------ | ------- |
| Flying Machine | Slim Fit Stretchable Jeans | ₹1829 | 3.6    | 152     |
| Flying Machine | Tapered Fit Jeans          | ₹1829 | 1.8    | 423 🔴  |
| Flying Machine | Relaxed Fit Jeans          | ₹2309 | 3.6    | 42–89   |
| Mufti          | Slim Fit Jeans             | ₹2149 | 3.4    | 5       |
| Turms          | 30 Days No Wash Jeans      | ₹3499 | 3.7    | 22      |


    
✅ Recommendation:

Immediate review required for Flying Machine Tapered Fit (1.8 rating, 423 reviews) — may harm brand reputation.

Audit Relaxed Fit and Slim Fit styles from Flying Machine — multiple models flagged.

Turms' ₹3499 jeans underperforming at premium price — consider customer feedback & return rate.

📌 Summary Table
| Category                   | Action                                                                 |
| -------------------------- | ---------------------------------------------------------------------- |
| 📈 Top Price Segment       | Promote ₹2000+ products with high ratings (esp. Levis, Flying Machine) |
| 🎯 High-Potential Brands   | Promote Calvin Klein, U.S. Polo, Comfits, Linaria                      |
| 🌟 Products to Feature     | Showcase Levi’s, Pepe, John Pride, Flying Machine Relaxed Fit          |
| ⚠️ Low-Performing Products | Investigate Flying Machine’s Slim & Tapered Fit Jeans                  |
| 💰 Discount Strategy       | Increase discounts for high-rated but under-discounted brands          |


In [13]:
promo_brands = brand_stats[
    (brand_stats['avg_discount'] < 0.45) &
    (brand_stats['avg_rating'] >= 4.0) &
    (brand_stats['num_ratings'] > 25000)
]


In [16]:
# ✅ Step 9: Export All Outputs to Excel
with pd.ExcelWriter("myntra_jeans_insights.xlsx") as writer:
    price_rating.to_excel(writer, sheet_name='Price vs Rating', index=False)
    promo_brands.to_excel(writer, sheet_name='Brands to Promote', index=False)
    top_rated.to_excel(writer, sheet_name='Top Rated Products', index=False)
    low_rating_high_price.to_excel(writer, sheet_name='Products to Investigate', index=False)
